In [1]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

# Chunking van tekst

## 1. Doelstellingen 

RAG komt erop neer dat een LLM wordt gevoed met extra context die relevant is voor een specifieke vraag, zodat het model beter in staat is om die vraag te beantwoorden. Die context kan echter niet onbeperkt groot zijn, om drie belangrijke redenen:

1. De meeste LLM’s hanteren een limiet op de grootte van de invoer die ze kunnen verwerken. Dit betekent dat zowel de vraag als de bijhorende context binnen een vast contextvenster moeten passen.
2. LLM’s zijn gevoelig voor het lost in the middle-fenomeen: ze besteden meer aandacht aan het begin en het einde van de invoer, en minder aan het minder van de invoer.
3. Het opslaan van context is een resource-intensief proces: hoe langer de context die moet worden opgeslagen, hoe meer tijd en middelen dit in beslag neemt.

Voor deze redenen is het nodig om de tekst afkomstig uit de PDF-bestanden te verdelen in *chunks* (= brokken). [Langchain](https://python.langchain.com/docs/introduction/) biedt zogenaamde *Text splitters* aan, die (platte) tekst opdelen in chunks volgens bepaalde strategieën. In dit Notebook bekijken we hoe bepaalde van deze splitters precies werken.

## 2. Methodologie 

We zullen een korte tekst van 559 tekens, bestaande uit twee paragrafen, door de volgende drie tekstsplitters laten verwerken:

1. `CharacterTextSplitter`: Deze splitter verdeelt de tekst in chunks op basis van een vooraf bepaalde chunkgrootte, zonder rekening te houden met de inhoud of structuur van de tekst.

2. `RecursiveCharacterTextSplitter`: Deze splitter hanteert een hiërarchische aanpak waarbij tekst wordt gesplitst volgens een voorkeursvolgorde van scheidingstekens. Er wordt pas naar het volgende niveau overgegaan als de chunk nog steeds groter is dan de opgegeven limiet. De volgorde is doorgaans: paragrafen (dubbele regeleinden (\n\n)), zinnen (enkele regeleinden (\n)), woorden (spaties (" ")) en uiteindelijk individuele tekens.

3. `SemanticChunker`: Deze splitter maakt gebruik van een embeddingmodel. Elke individuele zin wordt ge-embed als een vector. Zinnen die inhoudelijk sterk op elkaar lijken — met andere woorden waarvan de vectorrepresentaties dicht bij elkaar liggen — worden gegroepeerd. Deze groepen vormen de uiteindelijke chunks.

Vervolgens bekijken we de gegenereerde chunks en bespreken we onze waarnemingen in de besluit

## 3. Uittesten

In [2]:
txt = """Cloud computing maakt het mogelijk om rekenkracht en opslag via het internet te gebruiken zonder dat gebruikers lokale servers hoeven te beheren. Dit biedt schaalbaarheid, flexibiliteit en kostenbesparing, vooral voor bedrijven die snel willen groeien of wereldwijd actief zijn.

Virtualisatie stelt een enkele fysieke computer in staat om meerdere virtuele machines tegelijk te draaien. Elke virtuele machine functioneert als een aparte computer met zijn eigen besturingssysteem. Dit verhoogt de efficiëntie en maakt een betere benutting van hardware mogelijk."""

In [3]:
# Definiëren van de parameters van de text splitters

chunk_size = 300
chunk_overlap = 0
length_function = len

### 3.1. `CharacterTextSplitter`

In [4]:
character_splitter = CharacterTextSplitter(
    separator="",
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=length_function,
    is_separator_regex=True
)

charactersplitter_results = character_splitter.split_text(txt)

In [5]:
print(charactersplitter_results[0])

Cloud computing maakt het mogelijk om rekenkracht en opslag via het internet te gebruiken zonder dat gebruikers lokale servers hoeven te beheren. Dit biedt schaalbaarheid, flexibiliteit en kostenbesparing, vooral voor bedrijven die snel willen groeien of wereldwijd actief zijn.

Virtualisatie stelt


### 3.2. `RecursiveCharacterTextSplitter`

In [6]:
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=length_function,
    is_separator_regex=True
)

recursive_splitter_results = recursive_splitter.split_text(txt)

In [7]:
print(recursive_splitter_results[0])

Cloud computing maakt het mogelijk om rekenkracht en opslag via het internet te gebruiken zonder dat gebruikers lokale servers hoeven te beheren. Dit biedt schaalbaarheid, flexibiliteit en kostenbesparing, vooral voor bedrijven die snel willen groeien of wereldwijd actief zijn.


### 3.3. `SemanticChunker`

In [8]:
embedding_model = 'BAAI/bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

In [9]:
text_splitter = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount = 0.7,
)

semanticchunker_results = text_splitter.split_text(txt)

In [10]:
print(semanticchunker_results[0])

Cloud computing maakt het mogelijk om rekenkracht en opslag via het internet te gebruiken zonder dat gebruikers lokale servers hoeven te beheren.


## 4. Besluit

Bij het nader bekijken van de gegenereerde chunks, merken we het volgende:

- De `CharacterTextSplitter` splitst tekst op een eenvoudige, mechanische manier. Het knipt de tekst zodra de opgegeven limiet is bereikt, ongeacht of dat midden in een woord of zin gebeurt. Deze aanpak is snel en simpel, maar houdt geen rekening met de structuur of betekenis van de tekst. Hierdoor verliezen we potentieel belangrijke context. 

- De `RecursiveCharacterTextSplitter` probeert de tekst zo logisch mogelijk op te splitsen. Het gebruikt een lijst van scheidingstekens en zoekt telkens het meest geschikte punt om te splitsen, zolang het binnen de ingestelde lengte past. Pas als er geen logisch scheidingspunt gevonden wordt, splitst hij zoals de CharacterTextSplitter. Deze splitter is dus slimmer dan `CharacterTextSplitter`

- Hoewel de `SemanticChunker` in theorie veelbelovend is, stoten we in de praktijk op een belangrijk nadeel. Deze splitter verdeelt tekst vaak in kleine chunks, zoals individuele zinnen. Maar sommige zinnen vormen samen één betekenisvol geheel: ze bouwen voort op elkaar of verduidelijken elkaar. Wanneer zulke zinnen in aparte chunks terechtkomen, en slechts één ervan wordt opgehaald tijdens retrieval, gaat cruciale context verloren. Dit kan ertoe leiden dat het LLM het antwoord baseert op een te beperkt fragment en daardoor onnauwkeurig of onvolledig antwoordt.

## Intermezzo: Chunking strategie voor RAG-pijplijn

In dit [Notebook](./Tekstextractie_tekst.ipynb) hebben we gezien hoe tekst uit PDF-bestanden op een zeer envoudig manier ge-extraheerd kunnen worden met tools in de aard van [PyMuPDF](https://pymupdf.readthedocs.io/en/latest/). Echter, zorgt dit voor een moeilijkere chunkingproces. 

Wanneer we platte tekst uit PDF-bestanden extraheren, gaat de oorspronkelijke structuur verloren. Wat overblijft, is een zee van tekst zonder duidelijke indeling. Voor het aanmaken van chunks kunnen we dus niet vertrouwen op de oorspronkelijke structuur om samenhangende stukken te creëren. Daarom moeten we zorgvuldiger te werk gaan bij het chunkingproces. Als we zomaar chunkeren, bestaat het risico dat we suboptimale chunks aanmaken, waardoor de LLM geen volledig beeld krijgt van de inhoud.

Hier volgt de redenering achter de strategie die we zullen hanteren bij de RAG-pijplijn:

Eerst enkele parameters oplijsten:

- De meeste text splitters van Langchain hebben een parameter waarmee je de lengte van de chunks bepaalt. Deze wordt uitgedrukt in characters, terwijl de contextlengte van taalmodellen wordt aangegeven in tokens. We moeten dus een manier hebben om tokens naar characters (en omgekeerd) te vertalen. In de praktijk wordt aangenomen dat één token gemiddeld overeenkomt met vier characters. We zullen dit uitgangspunt hanteren.
- [Hier](../1.%20LLM/QuantFactory_Aya-23-8B-GGUF.ipynb) onderzochten we hoe geschikt het model `QuantFactory/aya-23-8B-GGUF` is qua Nederlandstalige vaardigheden en uitvoeringstijd. Aangezien we verder zullen werken met dit model, moeten we zijn contextvenster in rekening brengen. Volgens de [modelkaart](https://huggingface.co/QuantFactory/aya-23-8B-GGUF) heeft dit model een contextvenster van 8192 tokens, wat overeenkomt met ongeveer 32 768 characters.
- Characters vormen samen woorden. Voor de menselijke geest is het makkelijker om in woorden te redeneren. Het gemiddeld aantal letters per Nederlands (en ook Engels) woord is ongeveer vijf (zie [hier](https://pubmed.ncbi.nlm.nih.gov/33910411/)). Dit betekent dat 32 768 characters ongeveer 6554 woorden vertegenwoordigen.

Samenvattend: de vraag van de gebruiker, de instructies voor de LLM en de geselecteerde context moeten samen binnen 6554 woorden passen.

Daarnaast geloven we dat de structuur van een tekst vaak ook de onderliggende semantiek weerspiegelt: zinnen binnen een paragraaf behandelen meestal hetzelfde kernidee, en paragrafen binnen dezelfde sectie hebben doorgaans betrekking op hetzelfde onderwerp. Daarom kiezen we ervoor om onze chunks zo te maken dat (hopelijk) paragrafen samenblijven:

- Een gemiddelde paragraaf bevat tussen 100 en 200 woorden. We nemen de bovengrens van 200 woorden per paragraaf als referentie.
- Twee opeenvolgende paragrafen kunnen inhoudelijk sterk verbonden zijn. Om de LLM een ruimer contextbeeld te geven, besluiten we dat een chunk idealiter twee opeenvolgende paragrafen moet bevatten. Elke chunk zal dus ongeveer 400 woorden omvatten.
- 400 woorden → ongeveer 2000 characters → ongeveer 500 tokens.

Met deze chunkgrootte behouden we voldoende speling binnen het contextvenster van 8192 tokens: er blijven ongeveer 7692 tokens over voor de gebruikersvraag, de instructies en eventuele grotere chunks indien het nodig blijkt te zijn.